In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [2]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [3]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

4472
16986


In [29]:
from keras.layers import Dropout
from keras.layers import GRU
from keras.layers import Activation, Dense
from keras.activations import sigmoid

#vocabulary_size = 3800     # give your vocabulary size here (largest word ID in the input)
#embedding_dim = 32        # give your embedding dimension here (e.g. 100)

model = Sequential()
model.add(Embedding(3800, 32, input_length = X.shape[1]))
#model.add(Embedding(3800, 32))
model.add(Dropout(0.2))
#model.add(GRU(32, input_shape=(7188, 28)))
model.add(GRU(32))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

#model = Sequential()
#model.add(Embedding(vocabulary_size, embedding_dim, input_shape=(90582, 517)))
#model.add(GRU(512, return_sequences=True))
#model.add(Dropout(0.2))
#model.add(GRU(512, return_sequences=True))
#model.add(Dropout(0.2))
#model.add(TimeDistributedDense(1))
#model.add(Activation('softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
#model.fit(x_pad, y_pad, batch_size=128, nb_epoch=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 28, 32)            121600    
_________________________________________________________________
dropout_8 (Dropout)          (None, 28, 32)            0         
_________________________________________________________________
gru_4 (GRU)                  (None, 32)                6240      
_________________________________________________________________
dense_7 (Dense)              (None, 256)               8448      
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 136,545
Trainable params: 136,545
Non-trainable params: 0
_________________________________________________________________
None

In [30]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

((7188, 28), (7188, 2))
((3541, 28), (3541, 2))


In [38]:
import tensorflow as tf
batch_size = 32
epochs = 7
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train = tf.reshape(1, [-1, 7188, 28])

#binary_crossentropy,categorical_crossentropy
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size= batch_size, epochs=epochs,verbose=2)

ValueError: Dimension size must be evenly divisible by 201264 but is 1 for 'Reshape_4' (op: 'Reshape') with input shapes: [], [3] and with input tensors computed as partial shapes: input[1] = [?,7188,28].

In [9]:
from keras.callbacks import EarlyStopping

#trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
#X_train = X.reshape(1, 3, 3) # X.reshape(samples, timesteps, features)
#X_train = np.reshape(1, 1, 1)
#X_validate[x].reshape(1,X_test.shape[1])

#X_train = np.reshape(-1, 1, 1)
#print(X_train)

es=EarlyStopping(monitor='val_acc', patience=5)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 32
epochs = 7
model.fit(X_train, Y_train, validation_split=0.1, batch_size= batch_size, epochs=epochs, callbacks=[es],shuffle=True)

ValueError: Error when checking input: expected embedding_1_input to have 2 dimensions, but got array with shape (7188, 28, 7188)